# Pipeline try

We have selected some PDF samples with already-encoded text to test a complete pipeline worflow until the structured data extraction

## MLFLow experiment setup

Be sure to have run the mlflow server with this command at in the `prompt_enhancing/` directory

```sh
just serve-tracer
```

In [ ]:
import mlflow
import pandas as pd
from archaeo_super_prompt.env import getenv_or_throw

EXP_NAME = "NAIVE CHUNK SELECTION VIZU"
mlflow.set_tracking_uri(f"http://{getenv_or_throw("MLFLOW_HOST")}:{getenv_or_throw("MLFLOW_PORT")}")
mlflow.set_experiment(EXP_NAME)
mlflow.dspy.autolog()

pd.set_option('display.max_columns', None)

## Sample selection

In [ ]:
from pathlib import Path

from archaeo_super_prompt.dataset.load import MagohDataset
from archaeo_super_prompt.types.pdfpaths import buildPdfPathDataset

ds = MagohDataset(200, 0.8, True)
selected_ids = {
31049, 30913
}

selected_files = [
    (31049, Path(".cache/pdfs/31049/Relazione_storica_Pasquinucci.pdf").resolve()),
    (30913, Path(".cache/pdfs/30913/Relazione_assistenza.pdf").resolve()),
]

## Pipeline run

We use a dataframe-suitable version of the scikit-learn pipelines to pipe each module in this order :

- ocr
- layout text reading + chunking
- strucured data extraction

The LLM calls are traced by the MLFlow intergration and are viewable within links displayed by the cell below.

In [ ]:
from feature_engine.pipeline import Pipeline
from typing import cast
from archaeo_super_prompt.pdf_to_text import OCR_Transformer, TextExtractor
from archaeo_super_prompt.main_transformer import MagohDataExtractor
import archaeo_super_prompt.visualization.mlflow_logging as mmlflow

pipeline = Pipeline(
    [
        ("ocr", OCR_Transformer()),
        ("pdf_reader", TextExtractor()),
        ("extractor", MagohDataExtractor()),
    ]

)
extraction_model = cast(MagohDataExtractor, pipeline.named_steps["extractor"])
inputs = buildPdfPathDataset(selected_files)

with mlflow.start_run():
    mmlflow.save_models(pipeline, selected_files[0])
    score_value = pipeline.score(inputs, ds)
    score_results = extraction_model.score_results
    mmlflow.save_metric_scores(score_value, score_results)
    mmlflow.save_table_in_artifacts(score_results)

## Evaluation result inspection

In [ ]:
from archaeo_super_prompt.visualization.display_fields import display_results, run_display_server

display_results(score_results)

In [ ]:
run_display_server()